In [1]:
import sys
import os
sys.path.append('/usr0/home/drschwar/src/bert_erp/')
import json
import dataclasses

import numpy as np

from bert_erp_paths import Paths
from run_regression import named_variations
from analysis import print_variation_results, sentence_predictions
from bert_erp_datasets import read_natural_story_codings

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


In [2]:
paths = Paths()
id_to_coding = read_natural_story_codings(paths.natural_stories_path)

In [3]:
paths = Paths()
variation = 'nat_stories'
training_variations, settings, num_runs, _ = named_variations(variation)
for training_variation in training_variations:
    training_variation = training_variation
    print_variation_results(paths, variation, training_variation, num_runs)

Variation: ns_spr
name             mse        pove        povu         var
ns_spr      0.823998    0.216402    0.783598     1.05355


Variation: elan, epnp, lan, n400, ns_spr, p600, pnp
name             mse        pove        povu         var
ns_spr      0.927642    0.117933    0.882067     1.05355
elan        0.815637    0.190526    0.809474     1.01339
lan         0.649163    0.362257    0.637743     1.02386
n400         0.67574    0.307534    0.692466    0.982425
epnp        0.619011    0.379519    0.620481     1.00521
p600        0.660102    0.324835    0.675165    0.985622
pnp         0.628112    0.357842    0.642158    0.984811


Variation: elan, epnp, lan, n400, p600, pnp
name             mse        pove        povu         var
elan          0.8149    0.191144    0.808856     1.01339
lan         0.651539    0.359966    0.640034     1.02386
n400        0.676853    0.305853    0.694147    0.982425
epnp        0.619381     0.37919     0.62081     1.00521
p600        0.660762    0.3

In [4]:
sentence_outputs = sentence_predictions(paths, variation, training_variations[1], num_runs)

In [7]:
erp_values = dict()
data_key = 'natural_stories'
for erp in ('elan', 'lan', 'n400', 'epnp', 'p600', 'pnp'):
    erp_values[erp] = dict()
    for key in sorted(sentence_outputs[erp][data_key]):
        maxes = list()
        for idx in range(len(sentence_outputs[erp][data_key][key])):
            predictions = np.array(sentence_outputs[erp][data_key][key][idx]['prediction'])
            maxes.append(np.max(np.abs(predictions)))
        mean_max = np.mean(maxes)
        erp_values[erp][key] = mean_max
        
for erp in erp_values:
    sorted_erp_keys = list(sorted(erp_values[erp], key=lambda k: -erp_values[erp][k]))[:10]
    print(erp)
    for k in sorted_erp_keys:
        coding = dataclasses.asdict(id_to_coding[k])
        text = ''
        outputs = list()
        for field in coding:
            if field == 'sentence':
                text = coding[field]
            elif field == 'sentence_number':
                continue
            elif isinstance(coding[field], int) and coding[field] > 0:
                outputs.append('{}:{}'.format(field, coding[field]))
        print('{} {}'.format(text, ', '.join(outputs)))
    print('')
    print('')

elan
As the car pulled up outside the school, which looked like a large shadow in the darkness of a November evening, Lucy felt a shiver run down her spine. source_relative_clause_non_restrictive:1, idiom:1
Those shoes, they shined really brightly, and I liked looking at the bones in my feet, which I had never seen before, through a special x-ray machine they had in the shoe store that made the bones in your feet look green. object_relative_clause_non_restrictive:2, noun_phrase_sentence_ambiguity:1, topicalization:1
Some of the drops that were the most fortunate, including Aqua, fell on a mountainside, and down the rocky cliff they ran. complementizer_phrase_conjunctions:1, source_relative_clause_restrictive:1, parenthetical:1
She was the first performer. 
Lucy, who had started receiving these messages a few weeks ago, tapped the mobile screen to make the message disappear from view. source_relative_clause_non_restrictive:1, non_local_subject_verb:1
One of the popular girls, Lucy had a